# Chapter 7: Using Examples (Few-Shot Prompting)

- [Lesson](#lesson)
- [Exercises](#exercises)
- [Example Playground](#example-playground)

## Setup

Run the following setup cell to load your API key and establish the `get_completion` helper function.

In [ ]:
!pip install openai python-dotenv

# Import python's built-in regular expression library
import re
from openai import OpenAI
from dotenv import load_dotenv
import os

# Load environment variables from .env file
load_dotenv()

# Access variables
API_KEY = os.getenv("API_KEY")
BASE_URL = "https://api.deepseek.com"
MODEL_NAME = "deepseek-chat"

# Store the variables for use across notebooks
%store API_KEY
%store BASE_URL
%store MODEL_NAME

# For normal API requests
regular_client = OpenAI(
    api_key=API_KEY,
    base_url=BASE_URL
)

# For beta features like chat prefix completion
beta_client = OpenAI(
    api_key=API_KEY,
    base_url="https://api.deepseek.com/beta"
)

# New argument added for prefill text, with a default value of an empty string
def get_completion(prompt: str, system_prompt="", prefill=""):
    messages = []
    if system_prompt:
        messages.append({"role": "system", "content": system_prompt})
    
    messages.append({"role": "user", "content": prompt})
    
    # Use regular client by default
    client_to_use = regular_client
    
    # Only add assistant message if prefill is not empty
    if prefill:
        # For prefill, we need to use the beta endpoint and set prefix=True
        client_to_use = beta_client
        messages.append({"role": "assistant", "content": prefill, "prefix": True})
    
    response = client_to_use.chat.completions.create(
        model=MODEL_NAME,
        messages=messages,
        max_tokens=2000,
        temperature=0.0,
        stream=False
    )
    
    return response.choices[0].message.content

---

## Lesson

**Giving models examples of how you want them to behave (or how you want them not to behave) is extremely effective** for:
- Getting the right answer
- Getting the answer in the right format

This sort of prompting is also called "**few shot prompting**". You might also encounter the phrase "zero-shot" or "n-shot" or "one-shot". The number of "shots" refers to how many examples are used within the prompt.

### Examples

Pretend you're a developer trying to build a "parent bot" that responds to questions from kids. **By default, the model's response might be quite formal and robotic**. This is going to break a child's heart.

In [2]:
# Prompt
PROMPT = "Will Santa bring me presents on Christmas?"

# Print the model's response
print(get_completion(PROMPT))

Santa Claus is a beloved figure in Christmas traditions, and according to the stories, he brings presents to children who have been nice throughout the year. If you've been kind, helpful, and well-behaved, there's a good chance Santa will visit you on Christmas Eve and leave some presents under the tree! 

Of course, the spirit of Christmas is also about giving, sharing, and spending time with loved ones. So, whether or not Santa brings you presents, the most important thing is to enjoy the holiday season with joy and gratitude. 🎅🎄✨


You could take the time to describe your desired tone, but it's much easier just to **give the model a few examples of ideal responses**.

In [3]:
# Prompt
PROMPT = """Please complete the conversation by writing the next line, speaking as "A".
Q: Is the tooth fairy real?
A: Of course, sweetie. Wrap up your tooth and put it under your pillow tonight. There might be something waiting for you in the morning.
Q: Will Santa bring me presents on Christmas?"""

# Print the model's response
print(get_completion(PROMPT))

A: Absolutely! Santa loves bringing presents to good boys and girls. Just make sure to leave out some cookies and milk for him, and maybe a carrot for the reindeer. He’ll be so happy to visit you!


In the following formatting example, we could walk the model step by step through a set of formatting instructions on how to extract names and professions and then format them exactly the way we want, or we could just **provide the model with some correctly-formatted examples and it can extrapolate from there**. Note the `<individuals>` in the `assistant` turn to start the model off on the right foot.

In [11]:
# Prompt template with a placeholder for the variable content
PROMPT = """Silvermist Hollow, a charming village, was home to an extraordinary group of individuals.
Among them was Dr. Liam Patel, a neurosurgeon who revolutionized surgical techniques at the regional medical center.
Olivia Chen was an innovative architect who transformed the village's landscape with her sustainable and breathtaking designs.
The local theater was graced by the enchanting symphonies of Ethan Kovacs, a professionally-trained musician and composer.
Isabella Torres, a self-taught chef with a passion for locally sourced ingredients, created a culinary sensation with her farm-to-table restaurant, which became a must-visit destination for food lovers.
These remarkable individuals, each with their distinct talents, contributed to the vibrant tapestry of life in Silvermist Hollow.
<individuals>
1. Dr. Liam Patel [NEUROSURGEON]
2. Olivia Chen [ARCHITECT]
3. Ethan Kovacs [MUSICIAN AND COMPOSER]
4. Isabella Torres [CHEF]
</individuals>

At the heart of the town, Chef Oliver Hamilton has transformed the culinary scene with his farm-to-table restaurant, Green Plate. Oliver's dedication to sourcing local, organic ingredients has earned the establishment rave reviews from food critics and locals alike.
Just down the street, you'll find the Riverside Grove Library, where head librarian Elizabeth Chen has worked diligently to create a welcoming and inclusive space for all. Her efforts to expand the library's offerings and establish reading programs for children have had a significant impact on the town's literacy rates.
As you stroll through the charming town square, you'll be captivated by the beautiful murals adorning the walls. These masterpieces are the work of renowned artist, Isabella Torres, whose talent for capturing the essence of Riverside Grove has brought the town to life.
Riverside Grove's athletic achievements are also worth noting, thanks to former Olympic swimmer-turned-coach, Marcus Jenkins. Marcus has used his experience and passion to train the town's youth, leading the Riverside Grove Swim Team to several regional championships.
<individuals>
1. Oliver Hamilton [CHEF]
2. Elizabeth Chen [LIBRARIAN]
3. Isabella Torres [ARTIST]
4. Marcus Jenkins [COACH]
</individuals>

Oak Valley, a charming small town, is home to a remarkable trio of individuals whose skills and dedication have left a lasting impact on the community.
At the town's bustling farmer's market, you'll find Laura Simmons, a passionate organic farmer known for her delicious and sustainably grown produce. Her dedication to promoting healthy eating has inspired the town to embrace a more eco-conscious lifestyle.
In Oak Valley's community center, Kevin Alvarez, a skilled dance instructor, has brought the joy of movement to people of all ages. His inclusive dance classes have fostered a sense of unity and self-expression among residents, enriching the local arts scene.
Lastly, Rachel O'Connor, a tireless volunteer, dedicates her time to various charitable initiatives. Her commitment to improving the lives of others has been instrumental in creating a strong sense of community within Oak Valley.
Through their unique talents and unwavering dedication, Laura, Kevin, and Rachel have woven themselves into the fabric of Oak Valley, helping to create a vibrant and thriving small town."""

# Prefill for the model's response
PREFILL = "<individuals>"

# Print the model's response
print("--------------------------- Full prompt with variable substutions ---------------------------")
print("USER TURN:")
print(PROMPT)
print("\nASSISTANT TURN:")
print(PREFILL)
print("\n------------------------------------- Model's response -------------------------------------")
print(get_completion(PROMPT, prefill=PREFILL))

--------------------------- Full prompt with variable substutions ---------------------------
USER TURN:
Silvermist Hollow, a charming village, was home to an extraordinary group of individuals.
Among them was Dr. Liam Patel, a neurosurgeon who revolutionized surgical techniques at the regional medical center.
Olivia Chen was an innovative architect who transformed the village's landscape with her sustainable and breathtaking designs.
The local theater was graced by the enchanting symphonies of Ethan Kovacs, a professionally-trained musician and composer.
Isabella Torres, a self-taught chef with a passion for locally sourced ingredients, created a culinary sensation with her farm-to-table restaurant, which became a must-visit destination for food lovers.
These remarkable individuals, each with their distinct talents, contributed to the vibrant tapestry of life in Silvermist Hollow.
<individuals>
1. Dr. Liam Patel [NEUROSURGEON]
2. Olivia Chen [ARCHITECT]
3. Ethan Kovacs [MUSICIAN AND C

If you would like to experiment with the lesson prompts without changing any content above, scroll all the way to the bottom of the lesson notebook to visit the [**Example Playground**](#example-playground).

---

## Exercises
- [Exercise 7.1 - Email Formatting via Examples](#exercise-71---email-formatting-via-examples)

### Exercise 7.1 - Email Formatting via Examples
We're going to redo Exercise 6.2, but this time, we're going to edit the `PROMPT` to use "few-shot" examples of emails + proper classification (and formatting) to get the model to output the correct answer. We want the *last* letter of the model's output to be the letter of the category.

Refer to the comments beside each email in the `EMAILS` list if you forget which letter category is correct for each email.

Remember that these are the categories for the emails:										
- (A) Pre-sale question
- (B) Broken or defective item
- (C) Billing question
- (D) Other (please explain)								

In [13]:
# Prompt template with a placeholder for the variable content
PROMPT = """Classify the following email into one or more of the categories:

- **A**: Product question (about features, capabilities, usage, etc.)  
- **B**: Broken or defective item  
- **C**: Billing question  
- **D**: Other (for unusual or non-standard inquiries)  

**Instructions**:  
- If the email fits into multiple categories, list all applicable categories separated by commas.  
- Specifically, if a product question (A) is about an unusual or non-standard use, also classify it as D.  

**Examples**:  

<examples>  
<example>  
<email>Is the Mixmaster4000 compatible with 220V power outlets? I live in Europe and want to make sure it will work here.</email>  
<category>A</category>  
</example>  

<example>  
<email>Can I use the Mixmaster4000 to mix dough for bread?</email>  
<category>A</category>  
</example>  

<example>  
<email>Can I use my Mixmaster4000 to mix paint?</email>  
<category>A, D</category>  
</example>  

<example>  
<email>The Mixmaster4000 I received last week stopped working after just two uses. The motor won’t turn on anymore. Can you help?</email>  
<category>B</category>  
</example>  

<example>  
<email>I canceled my subscription last month, but I’m still being charged. Can you stop the charges and refund me for the last payment?</email>  
<category>C</category>  
</example>  

<example>  
<email>My Mixmaster4000 started smoking after I used it to mix thick cookie dough. And I just found out I was charged twice.</email>  
<category>B, C</category>  
</example>  

<example>  
<email>How did I get here I am not good with computer. Halp.</email>  
<category>D</category>  
</example>  
</examples>  

Now, classify the following email:  

<email>  
{email}  
</email>  

**Output Instructions**: Output only the category letters (e.g., "A" or "B, C").  
        """

# Prefill for the model's response
PREFILL = ""

# Variable content stored as a list
EMAILS = [
    "Hi -- My Mixmaster4000 is producing a strange noise when I operate it. It also smells a bit smoky and plasticky, like burning electronics.  I need a replacement.", # (B) Broken or defective item
    "Can I use my Mixmaster 4000 to mix paint, or is it only meant for mixing food?", # (A) Pre-sale question OR (D) Other (please explain)
    "I HAVE BEEN WAITING 4 MONTHS FOR MY MONTHLY CHARGES TO END AFTER CANCELLING!!  WTF IS GOING ON???", # (C) Billing question
    "How did I get here I am not good with computer.  Halp." # (D) Other (please explain)
]

# Correct categorizations stored as a list of lists to accommodate the possibility of multiple correct categorizations per email
ANSWERS = [
    ["B"],
    ["A","D"],
    ["C"],
    ["D"]
]

# Iterate through list of emails
for i,email in enumerate(EMAILS):
    
    # Substitute the email text into the email placeholder variable
    formatted_prompt = PROMPT.format(email=email)
   
    # Get the model's response
    response = get_completion(formatted_prompt, prefill=PREFILL)

    # Grade the model's response
    grade = any([bool(re.search(ans, response[-1])) for ans in ANSWERS[i]])
    
    # Print the model's response
    print("--------------------------- Full prompt with variable substutions ---------------------------")
    print("USER TURN")
    print(formatted_prompt)
    print("\nASSISTANT TURN")
    print(PREFILL)
    print("\n------------------------------------- Model's response -------------------------------------")
    print(response)
    print("\n------------------------------------------ GRADING ------------------------------------------")
    print("This exercise has been correctly solved:", grade, "\n\n\n\n\n\n")

--------------------------- Full prompt with variable substutions ---------------------------
USER TURN
Classify the following email into one or more of the categories:

- **A**: Product question (about features, capabilities, usage, etc.)  
- **B**: Broken or defective item  
- **C**: Billing question  
- **D**: Other (for unusual or non-standard inquiries)  

**Instructions**:  
- If the email fits into multiple categories, list all applicable categories separated by commas.  
- Specifically, if a product question (A) is about an unusual or non-standard use, also classify it as D.  

**Examples**:  

<examples>  
<example>  
<email>Is the Mixmaster4000 compatible with 220V power outlets? I live in Europe and want to make sure it will work here.</email>  
<category>A</category>  
</example>  

<example>  
<email>Can I use the Mixmaster4000 to mix dough for bread?</email>  
<category>A</category>  
</example>  

<example>  
<email>Can I use my Mixmaster4000 to mix paint?</email>  
<cat

❓ If you want a hint, run the cell below!

In [ ]:
from hints import exercise_7_1_hint; print(exercise_7_1_hint)

Still stuck? Run the cell below for an example solution.

In [ ]:
from hints import exercise_7_1_solution; print(exercise_7_1_solution)

### Congrats!

If you've solved all exercises up until this point, you're ready to move to the next chapter. Happy prompting!

---

## Example Playground

This is an area for you to experiment freely with the prompt examples shown in this lesson and tweak prompts to see how it may affect the model's responses.

In [ ]:
# Prompt
PROMPT = "Will Santa bring me presents on Christmas?"

# Print the model's response
print(get_completion(PROMPT))

In [ ]:
# Prompt
PROMPT = """Please complete the conversation by writing the next line, speaking as "A".
Q: Is the tooth fairy real?
A: Of course, sweetie. Wrap up your tooth and put it under your pillow tonight. There might be something waiting for you in the morning.
Q: Will Santa bring me presents on Christmas?"""

# Print the model's response
print(get_completion(PROMPT))

In [8]:
# Prompt template with a placeholder for the variable content
PROMPT = """
Extract the individuals from the story and enclose them in <individuals></individuals> tags with their roles as in the examples provided.
**Examples:**
Silvermist Hollow, a charming village, was home to an extraordinary group of individuals.
Among them was Dr. Liam Patel, a neurosurgeon who revolutionized surgical techniques at the regional medical center.
Olivia Chen was an innovative architect who transformed the village's landscape with her sustainable and breathtaking designs.
The local theater was graced by the enchanting symphonies of Ethan Kovacs, a professionally-trained musician and composer.
Isabella Torres, a self-taught chef with a passion for locally sourced ingredients, created a culinary sensation with her farm-to-table restaurant, which became a must-visit destination for food lovers.
These remarkable individuals, each with their distinct talents, contributed to the vibrant tapestry of life in Silvermist Hollow.
<individuals>
1. Dr. Liam Patel [NEUROSURGEON]
2. Olivia Chen [ARCHITECT]
3. Ethan Kovacs [MUSICIAN AND COMPOSER]
4. Isabella Torres [CHEF]
</individuals>

At the heart of the town, Chef Oliver Hamilton has transformed the culinary scene with his farm-to-table restaurant, Green Plate. Oliver's dedication to sourcing local, organic ingredients has earned the establishment rave reviews from food critics and locals alike.
Just down the street, you'll find the Riverside Grove Library, where head librarian Elizabeth Chen has worked diligently to create a welcoming and inclusive space for all. Her efforts to expand the library's offerings and establish reading programs for children have had a significant impact on the town's literacy rates.
As you stroll through the charming town square, you'll be captivated by the beautiful murals adorning the walls. These masterpieces are the work of renowned artist, Isabella Torres, whose talent for capturing the essence of Riverside Grove has brought the town to life.
Riverside Grove's athletic achievements are also worth noting, thanks to former Olympic swimmer-turned-coach, Marcus Jenkins. Marcus has used his experience and passion to train the town's youth, leading the Riverside Grove Swim Team to several regional championships.
<individuals>
1. Oliver Hamilton [CHEF]
2. Elizabeth Chen [LIBRARIAN]
3. Isabella Torres [ARTIST]
4. Marcus Jenkins [COACH]
</individuals>

**Story**
Oak Valley, a charming small town, is home to a remarkable trio of individuals whose skills and dedication have left a lasting impact on the community.
At the town's bustling farmer's market, you'll find Laura Simmons, a passionate organic farmer known for her delicious and sustainably grown produce. Her dedication to promoting healthy eating has inspired the town to embrace a more eco-conscious lifestyle.
In Oak Valley's community center, Kevin Alvarez, a skilled dance instructor, has brought the joy of movement to people of all ages. His inclusive dance classes have fostered a sense of unity and self-expression among residents, enriching the local arts scene.
Lastly, Rachel O'Connor, a tireless volunteer, dedicates her time to various charitable initiatives. Her commitment to improving the lives of others has been instrumental in creating a strong sense of community within Oak Valley.
Through their unique talents and unwavering dedication, Laura, Kevin, and Rachel have woven themselves into the fabric of Oak Valley, helping to create a vibrant and thriving small town."""

# Prefill for the model's response
PREFILL = "<individuals>"

# Print the model's response
print("--------------------------- Full prompt with variable substutions ---------------------------")
print("USER TURN:")
print(PROMPT)
print("\nASSISTANT TURN:")
print(PREFILL)
print("\n------------------------------------- Model's response -------------------------------------")
print(get_completion(PROMPT, prefill=PREFILL))

--------------------------- Full prompt with variable substutions ---------------------------
USER TURN:

Extract the individuals from the story and enclose them in <individuals></individuals> tags with their roles as in the examples provided.
**Examples:**
Silvermist Hollow, a charming village, was home to an extraordinary group of individuals.
Among them was Dr. Liam Patel, a neurosurgeon who revolutionized surgical techniques at the regional medical center.
Olivia Chen was an innovative architect who transformed the village's landscape with her sustainable and breathtaking designs.
The local theater was graced by the enchanting symphonies of Ethan Kovacs, a professionally-trained musician and composer.
Isabella Torres, a self-taught chef with a passion for locally sourced ingredients, created a culinary sensation with her farm-to-table restaurant, which became a must-visit destination for food lovers.
These remarkable individuals, each with their distinct talents, contributed to the